In [1]:
import os

In [2]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification'

In [5]:
import tensorflow as tf 

2024-12-19 14:05:49.430332: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_parameters: dict
    parameters_image_size: list
    parameters_batch_size: int
    

In [8]:
from classifier.constants import *
from classifier.utils.common_functions import read_yaml, create_dirctories, save_json


2024-12-19 14:05:57,841 - classifier - INFO - welcome tinashe


In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            parameters_filepath= PARAMETERS_FILE_PATH
            ):
       
        self.config= read_yaml(config_filepath)
        self.parameters= read_yaml(parameters_filepath)
        create_dirctories([self.config.artifacts_roots])

    def get_validation_config(self)->EvaluationConfig:
        eval_config=EvaluationConfig(
            path_of_model= "artifacts/training/model.keras",
            training_data= "artifacts/data_ingestion/chicken_images",
            all_parameters= self.parameter,
            parameters_image_size= self.parameters.IMAGE_SIZE,
            parameters_batch_size= self.parameters.BATCH_SIZE
            
        )

        return eval_config

In [10]:
from urllib.parse import urlparse 

In [11]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config= config

    def _valid_generator(self):


        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split= 0.30
        )

        dataflow_kwargs= dict(
            target_size= self.config.parameters_image_size[:-1],
            batch_size= self.config.parameters_batch_size,
            interpolation= "bilinear"

        )



        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self._valid_generator= valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle= False,
            **dataflow_kwargs

        )



    @staticmethod
    def load_model(path: Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
        
    def evaluation(self):
        self.model= self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=self.model.evaluate(self._valid_generator)

    def save_score(self):
            scores= {"loss":self.score[0], "accuracy": self.score[1]}
            save_json(path=Path("scores.json"), data= scores)






    




In [12]:
try:
    config= ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

2024-12-19 14:05:58,309 - classifier - INFO - yaml file: config/config.yaml loaded succefully
2024-12-19 14:05:58,341 - classifier - INFO - yaml file: parameters.yaml loaded succefully
2024-12-19 14:05:58,359 - classifier - INFO - created a directory at: artifacts
Found 116 images belonging to 2 classes.


/home/tinashe/.conda/envs/chicken_tools/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-12-19 14:06:04.371989: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-12-19 14:06:05.871484: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2024-12-19 14:06:15.405211: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2024-12-19 14:06:15.647176: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2024-12-

8/8 ━━━━━━━━━━━━━━━━━━━━ 272s 29s/step - accuracy: 0.7504 - loss: 13.0292
2024-12-19 14:10:33,348 - classifier - INFO -  json file saved at:scores.json
